**Your First AI Agent: From Prompt to Action**

This notebook is your first step into building AI agents. An agent can do more than just respond to a prompt — it can take actions to find information or get things done.

❌ Note: Avoid using the Run all cells command as this can trigger a QPM limit resulting in 429 errors when calling the backing model. Suggested flow is to run each cell in order - one at a time

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
    
except Exception as e:
    print( f"🔑 Authentication Error:Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}" 
    )

✅ Gemini API key setup complete.


**Import ADK components**
Now, import the specific components you'll need from the **Agent Development Kit and the Generative AI library**. This keeps your code organized and ensures we have access to the necessary building blocks.

In [2]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types

print("✅ ADK components import Successfully.")


✅ ADK components import Successfully.


**Helper functions** 

We'll define some helper function. if you are running this outside the kaggle enviroment, you dont need to do this. 

In [3]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


**Configure Retry Options**

when working with LLMs,you may encounter transient error like rate limits or temporary service unavilablity.Retry options automatically handle these failures by retrying the request with exponential backoff.

In [4]:
retry_config = types.HttpRetryOptions(
    attempts = 5, # Maximum retry attempts
    exp_base = 7, # delay multiplier
    initial_delay = 1, #Initial delay before first retry(in secounds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

**Section 2:** **Your first AI Agent with ADK**

🤔 2.1 What is an AI Agent?
You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.
***Prompt -> LLM -> Text***

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.
***Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer** *








we'll build an agent that can take the action of searching Google. Let's see the difference!

**2.2 Define your agent**

We'll configure an Agent by setting its key properties, which tell it what to do and how to operate.

These are the main properties we'll set:
* name and description: A simple name and description to identify our agent.
* model: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
* instruction: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
* tools: A list of tools that the agent can use. To start, we'll give it the google_search tool, which lets it find up-to-date information online.

In [5]:
root_agent = Agent(
    name = "Rajab",
    model = Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description = "A simple agent that can answer general Questions.",
    instruction = " You are a helpful assintant. Use Google Search for current info or if unsure.",
    tools =[google_search]
    
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


**2.3 Run your agent**


Now it's time to bring your agent to life and send it a query. To do this, you need a Runner, which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

A.Create an *InMemoryRunner* and tell it to use our *root_agent*

In [6]:
runner = InMemoryRunner(agent = root_agent)
print("✅ Runner created.")

✅ Runner created.



Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as adk run, adk web, or adk api_server. To learn more, check out the documentation related to [runtime in ADK](http://).

b. 
Now you can call the **.run_debug()** method to send our prompt and get an answer.

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [7]:
resposnse = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
Rajab > The Agent Development Kit (ADK) from Google is a flexible and modular open-source framework designed to simplify the development, deployment, and orchestration of AI agents and multi-agent systems. It aims to make agent development more akin to traditional software development, offering precise control over agent behavior and orchestration. While optimized for Google's Gemini models and Vertex AI, ADK is model-agnostic and compatible with other frameworks. It allows developers to define agent logic, tools, and orchestration directly in code, which aids in debugging, versioning, and deployment.

The ADK SDK is currently available in the following languages:
*   Python
*   Go
*   Java

Google has indicated that support for more languages is coming soon.


**2.4 How does it work?**

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:
* The agent inspects and is aware of which tools it has available to use
* The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the ADK web UI, which we'll set up later in this notebook.

In [8]:
response = await runner.run_debug("What is the weather in London?")


 ### Continue session: debug_session_id

User > What is the weather in London?
Rajab > The weather in London is currently mostly cloudy with a temperature of 5°C (41°F). There is a 0% chance of rain. The humidity is around 85%.

The forecast for today, Monday, November 24, 2025, indicates light rain during the day and mostly cloudy conditions at night, with a 50% chance of rain during the day and a 20% chance at night. Temperatures are expected to be between 4°C (40°F) and 7°C (44°F).

Looking ahead, tomorrow, Tuesday, November 25, 2025, is expected to be sunny during the day and clear at night, with a 5% chance of rain during the day and a 5% chance of snow at night. Temperatures will range from 1°C (34°F) to 7°C (45°F).


In [9]:
response = await runner.run_debug("Who won the last soccer world cup?")


 ### Continue session: debug_session_id

User > Who won the last soccer world cup?
Rajab > Argentina won the last soccer World Cup in 2022. They defeated France in the final match, which ended in a 3-3 draw and was decided by a penalty shootout, with Argentina winning 4-2. This was Argentina's third World Cup title.


In [10]:
response = await runner.run_debug("What new movies are showing in theaters now?")


 ### Continue session: debug_session_id

User > What new movies are showing in theaters now?
Rajab > Here are some of the new movies currently showing in theaters:

*   **Wicked: For Good**
*   **Now You See Me: Now You Don't**
*   **The Running Man**
*   **Predator: Badlands**
*   **Sisu: Road to Revenge**
*   **Rental Family**
*   **Regretting You**
*   **Sarah's Oil**
*   **Keeper**
*   **Nuremberg**
*   **The Black Phone 2**
*   **Tron: Ares**
*   **Chainsaw Man - The Movie: Reze Arc (Dub)**

Some family-friendly options include:
*   **Zootopia 2**
*   **The Pout-Pout Fish** (scheduled for March 2026)

Please note that showtimes and availability can vary by location, so it's always a good idea to check your local theater listings.


**Section 3: Try the ADK Web Interface**

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

In [11]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



In [12]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [99]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http:/

Note: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.

**‼️ IMPORTANT**: DO NOT SHARE THE PROXY LINK with anyone - treat it as sensitive data as it contains your authentication token in the URL.
 

🚀 Building My First AI Agent Using Google’s ADK

Today, I completed an exciting step in my AI journey—creating my first AI Agent using the Agent Development Kit (ADK). This assignment helped me understand how agents go beyond simple LLM prompts and bring real reasoning + action capabilities into applications.

🔍 Importing ADK Components

I began by importing the required components from the ADK and Generative AI library. This ensures clean, modular code and gives access to all the essential building blocks needed to design intelligent agents.

🤖 What Exactly Is an AI Agent?

While traditional LLMs simply take a prompt and return text, an AI Agent is different. It can think, take actions, and observe results before generating a final answer.
Prompt → Agent → Thought → Action → Observation → Final Answer

This allows the agent to produce smarter, more accurate responses.

🛠️ Defining My Agent

I configured the agent with key parameters:

Name & Description – to identify the agent’s purpose

Model – used gemini-2.5-flash-lite for efficient reasoning

Instruction – the guiding prompt that defines the agent’s behavior

Tools – enabled the google_search tool so the agent can access real-time information

🎉 Final Result

I successfully built and ran the agent. The most exciting part was seeing it reason about what information it needed and automatically use tools to find answers. This highlighted the true strength of agent-based AI—not just responding, but acting.

This project gave me a solid foundation in AI Agent development, and I’m excited to keep exploring what’s possible with ADK!